In [1]:
!git clone https://github.com/MeesLursen/KernelLTL.git

Cloning into 'KernelLTL'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 248 (delta 7), reused 18 (delta 6), pack-reused 225 (from 1)
Receiving objects: 100% (248/248), 99.18 KiB | 923.00 KiB/s, done.
Resolving deltas: 100% (136/136), done.


In [2]:
!cd /kaggle/working/KernelLTL && git pull && cd ..

Already up to date.


In [3]:
import sys
sys.path.insert(0, "/kaggle/working/KernelLTL") 

In [4]:
!wget -qO- https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
!./bin/micromamba create -y -n kernelltl -c conda-forge python=3.13.7 pip
!./bin/micromamba run -n kernelltl pip install -r /kaggle/working/KernelLTL/requirements.txt

bin/micromamba
[+] 0.0s
[+] 0.1s
[+] 0.2s
conda-forge/linux-64 ━━━━━━━━━━━━━━━━━━━━━━━ 804.3kB /  47.8MB @   4.6MB/s  0.1s
conda-forge/noarch   ━╸━━━━━━━━━━━━━━━━━━━━━   2.8MB /  22.8MB @  16.0MB/s  0.1s[+] 0.3s
conda-forge/linux-64 ━╸━━━━━━━━━━━━━━━━━━━━━   5.6MB /  47.8MB @  19.9MB/s  0.2s
conda-forge/noarch   ━━━━━━━╸━━━━━━━━━━━━━━━   8.3MB /  22.8MB @  29.5MB/s  0.2s[+] 0.4s
conda-forge/linux-64 ━━╸━━━━━━━━━━━━━━━━━━━━   8.1MB /  47.8MB @  21.2MB/s  0.3s
conda-forge/noarch   ━━━━━━━━━━━╸━━━━━━━━━━━  12.5MB /  22.8MB @  32.7MB/s  0.3s[+] 0.5s
conda-forge/linux-64 ━━━━╸━━━━━━━━━━━━━━━━━━  12.5MB /  47.8MB @  25.7MB/s  0.4s
conda-forge/noarch   ━━━━━━━━━━━━━━━━╸━━━━━━  17.6MB /  22.8MB @  35.9MB/s  0.4s[+] 0.6s
conda-forge/linux-64 ━━━━━━━╸━━━━━━━━━━━━━━━  17.7MB /  47.8MB @  29.9MB/s  0.5s
conda-forge/noarch   ━━━━━━━━━━━━━━━━━━━━━╸━  22.6MB /  22.8MB @  38.2MB/s  0.5sconda-forge/noarch                                  22.8MB @  38.2MB/s  0.6s
[+] 0.7s
conda-forge/linux-64 ━━━━━━━╸━━

In [5]:
!cd /kaggle/working/KernelLTL && \
 PYTHONPATH=$PWD /kaggle/working/bin/micromamba run -n kernelltl \
 torchrun --nproc_per_node=1 -m train

/root/.local/share/mamba/envs/kernelltl/lib/python3.13/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 Tesla P100-PCIE-16GB which is of cuda capability 6.0.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (7.0) - (12.0)
    
  warnings.warn(
/root/.local/share/mamba/envs/kernelltl/lib/python3.13/site-packages/torch/cuda/__init__.py:304: UserWarning: 
    Please install PyTorch with a following CUDA
    configurations:  12.6 following instructions at
    https://pytorch.org/get-started/locally/
    
  warnings.warn(matched_cuda_warn.format(matched_arches))
/root/.local/share/mamba/envs/kernelltl/lib/python3.13/site-packages/torch/cuda/__init__.py:326: UserWarning: 
Tesla P100-PCIE-16GB with CUDA capability sm_60 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_70 sm_75 sm_80 sm_86 sm_90 sm_100 sm_120.
If you want to use the Tesla P100-PCIE-16GB GPU with PyTorch, p

In [6]:
import os, json, shutil

import os, shutil
os.makedirs("/root/.kaggle", exist_ok=True)
shutil.copy("/kaggle/input/kaggle-api-token/kaggle.json", "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 0o600)

from kaggle import api
ds_dir = "/kaggle/working/KernelLTL/ltl_model_outputs"
os.makedirs(ds_dir, exist_ok=True)

metadata = {
    "title": "LTL model outputs",
    "id": "tayhnd5sgdtjevyreuw/kernel-ltl-model-outputs",
    "licenses": [{"name": "CC0-1.0"}]
}
with open(os.path.join(ds_dir, "dataset-metadata.json"), "w") as f:
    json.dump(metadata, f, indent=2)

api.dataset_create_new(ds_dir)

{"ref": "", "url": "https://www.kaggle.com/", "status": "Error", "error": "Please upload at least one file", "invalidTags": []}

'''
import shutil, os

def kaggle_export(output_dir, zip_name):
    """Create a zip file and export your work in the kaggle working directory.
    Args:
        output_dir (str): The path to the directory to be zipped.
        zip_name (str): The name of the zip file to be created.
    Returns:
        None
    """
    if not os.path.exists(output_dir):
        print(f"[Error]: output directory '{output_dir}' does not exist")
    
    else:
        try:
            shutil.make_archive(zip_name, 'zip', output_dir)
            print(f"[Success]: Your zip file is successfully created!")

        except Exception as e:
            print(f"[Error]: An error occurred while creating zip: {e}")

kaggle_export("/kaggle/working/KernelLTL/ltl_model_outputs", "model_outputs")
'''

'''
import math
import os
import torch

# Hyperparameters
num_epochs = 10

learning_rate = 5e-5

T       = 20
AP      = 5
seed    = 1

eps     = 0.01
delta   = 1 - 0.99
N       = math.ceil((2 / eps**2) * math.log(2 / delta))

m       = 1024
    
# Create output directory
output_dir = "ltl_model_outputs"
os.makedirs(output_dir, exist_ok=True)
local_rank = int(os.environ.get("LOCAL_RANK", -1))
if local_rank != -1 and torch.cuda.is_available():
    torch.cuda.set_device(local_rank)
# os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
# torch.use_deterministic_algorithms(True)
'''

'''
from transformers import TrainingArguments, Trainer
from tokenizer_class import LTLTokenizer
from kernel_class import LTLKernel
from dataset_class import LTLDataset

# Initialize tokenizer (adjust n_ap based on your needs)
tokenizer = LTLTokenizer(n_ap=AP)

# Initialize kernel for semantic embeddings
kernel = LTLKernel(T, AP, seed)  # adjust T and AP as needed
kernel.sample_traces_kernel(N)  # adjust N based on your needs
kernel.sample_anchor_formulas_kernel(m)  # m should match model's n_embd
kernel.build_F()

# Create datasets
train_dataset = LTLDataset()
train_dataset.construct_dataset_from_kernel(
    kernel=kernel,
    k=78000,  # adjust dataset size as needed
    p_leaf=0.45,
    max_depth=1000,
    batch_size=10240
)

eval_dataset = LTLDataset()
eval_dataset.construct_dataset_from_kernel(
    kernel=kernel,
    k=1000,  # smaller validation set
    p_leaf=0.45,
    max_depth=1000,
    batch_size=10240
)
'''

'''
from model_class import LTLModel
from config_class import LTLConfig
from training_utils import SemanticEvaluationCallback

# Create model configuration and model
config = LTLConfig(
    n_embd=m  # must match kernel's anchor set size (m)
)

model = LTLModel(config, semantic_emb_dim=m)  # semantic_emb_dim must match kernel's anchor set size

# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_epochs,
    learning_rate=learning_rate,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=f"{output_dir}/logs",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    dataloader_num_workers=4,
    dataloader_pin_memory=True,
    ddp_find_unused_parameters=False
)

# Initialize callback
semantic_callback = SemanticEvaluationCallback(
    kernel=kernel,
    tokenizer=tokenizer
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=lambda batch : tokenizer.collate_batch(batch, model.config.n_positions),
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    callbacks=[semantic_callback]
)
'''

'''
# Train
trainer.train()

# Save final model
trainer.save_model(os.path.join(output_dir, "final_model"))
tokenizer.save_vocab(os.path.join(output_dir, "vocab.json"))
'''